In [1]:
!git clone https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory;pip install -e ".[torch,metrics]"
!pip install bitsandbytes>=0.39.0

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 15349, done.
remote: Counting objects: 100% (299/299), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 15349 (delta 186), reused 230 (delta 159), pack-reused 15050
Receiving objects: 100% (15349/15349), 221.57 MiB | 28.23 MiB/s, done.
Resolving deltas: 100% (11235/11235), done.
Obtaining file:///kaggle/working/LLaMA-Factory
  Installing build dependencies ... - \ | / done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... - done
  Preparing editable metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json

file_path = '/kaggle/working/LLaMA-Factory/data/dataset_info.json'

with open(file_path, 'r') as file:
    data = json.load(file)

# configure the MedQA and PubMedQA dataset information 
new_data = {
    "MedQA_train": {"file_name": "MedQA/train.json"},
    "MedQA_test": {"file_name": "MedQA/test.json"},
    "PubMedQA_pqal_train": {"file_name": "PubMedQA/pqal_train_set.json"},
    "PubMedQA_pqal_test": {"file_name": "PubMedQA/pqal_test_set.json"}
}

# update information in dataset_info.json file
data.update(new_data)
with open(file_path, 'w') as file:
    json.dump(data, file, indent=4)

In [3]:
!ls /kaggle/input/

medqa-pubmedqa


In [4]:
# create MedQA and PubMedQA directories
!mkdir -p /kaggle/working/LLaMA-Factory/data/MedQA
!mkdir -p /kaggle/working/LLaMA-Factory/data/PubMedQA

# copy MedQA dataset file
!cp /kaggle/input/medqa-pubmedqa/MedQA/train.json /kaggle/working/LLaMA-Factory/data/MedQA/
!cp /kaggle/input/medqa-pubmedqa/MedQA/test.json /kaggle/working/LLaMA-Factory/data/MedQA/

# copy PubMedQA dataset file
!cp /kaggle/input/medqa-pubmedqa/PubMedQA/pqal_train_set.json /kaggle/working/LLaMA-Factory/data/PubMedQA/
!cp /kaggle/input/medqa-pubmedqa/PubMedQA/pqal_test_set.json /kaggle/working/LLaMA-Factory/data/PubMedQA/

In [5]:
%cd LLaMA-Factory

/kaggle/working/LLaMA-Factory


In [6]:
# install packages for quantizaiton
!pip install auto_gptq>=0.5.0
!pip install optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.7/424.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.9 MB/s eta 0:00:00


max_example = 250

In [7]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 5e-05 \
    --num_train_epochs 3.0 \
    --max_samples 250 \
    --per_device_train_batch_size 6 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves\Qwen2-0.5B-Instruct\lora\train_2024-07-08-20-40-25 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 5 \
    --per_device_eval_batch_size 2 

2024-07-11 04:39:18.735627: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 04:39:18.735754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 04:39:18.884104: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
07/11/2024 04:39:35 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
tokenizer_config.json: 100%|███████████████| 1.29k/1.29k [00:00<00:00, 8.01MB/s]
vocab.json: 100%|██████████████████████████| 2.78M/2.78M [00:00<00:00, 35.9MB/s]
merges.txt: 100%|██████████████████████████

In [8]:
!llamafactory-cli train \
    --stage sft \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_test,PubMedQA_pqal_test \
    --cutoff_len 1024 \
    --max_samples 100 \
    --per_device_eval_batch_size 2 \
    --predict_with_generate True \
    --max_new_tokens 512 \
    --top_p 0.7 \
    --temperature 0.95 \
    --output_dir saves\Qwen2-0.5B-Instruct\lora\eval_2024-07-09-00-25-34 \
    --do_predict True \
    --adapter_name_or_path saves\Qwen2-0.5B-Instruct\lora\train_2024-07-08-20-40-25

2024-07-11 04:47:01.571791: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 04:47:01.571847: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 04:47:01.573349: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
07/11/2024 04:47:10 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: None
[INFO|tokenization_utils_base.py:2108] 2024-07-11 04:47:10,320 >> loading file vocab.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2-0.5B-Instruct/snapshots/c291d6fce4804a1d39305f388dd32897d1f7a

max_example=500

In [9]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 5e-05 \
    --num_train_epochs 3.0 \
    --max_samples 500 \
    --per_device_train_batch_size 6 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves\Qwen2-0.5B-Instruct\lora\train_2024-07-08-20-40-26 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 5 \
    --per_device_eval_batch_size 2 

2024-07-11 04:49:43.768470: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 04:49:43.768526: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 04:49:43.770092: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
07/11/2024 04:49:52 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2108] 2024-07-11 04:49:52,581 >> loading file vocab.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2-0.5B-Instruct/snapshots/c291d6fce4804a1d39305f388dd3

In [10]:
!llamafactory-cli train \
    --stage sft \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_test,PubMedQA_pqal_test \
    --cutoff_len 1024 \
    --max_samples 100 \
    --per_device_eval_batch_size 2 \
    --predict_with_generate True \
    --max_new_tokens 512 \
    --top_p 0.7 \
    --temperature 0.95 \
    --output_dir saves\Qwen2-0.5B-Instruct\lora\eval_2024-07-09-00-25-35 \
    --do_predict True \
    --adapter_name_or_path saves\Qwen2-0.5B-Instruct\lora\train_2024-07-08-20-40-26

2024-07-11 05:04:44.985601: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 05:04:44.985657: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 05:04:44.987127: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
07/11/2024 05:04:53 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: None
[INFO|tokenization_utils_base.py:2108] 2024-07-11 05:04:53,719 >> loading file vocab.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2-0.5B-Instruct/snapshots/c291d6fce4804a1d39305f388dd32897d1f7a

max_example=1000

In [11]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 5e-05 \
    --num_train_epochs 3.0 \
    --max_samples 1000 \
    --per_device_train_batch_size 6 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves\Qwen2-0.5B-Instruct\lora\train_2024-07-08-20-40-27 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 5 \
    --per_device_eval_batch_size 2 

2024-07-11 05:07:23.268568: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 05:07:23.268626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 05:07:23.270165: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
07/11/2024 05:07:31 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2108] 2024-07-11 05:07:32,002 >> loading file vocab.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2-0.5B-Instruct/snapshots/c291d6fce4804a1d39305f388dd3

In [12]:
!llamafactory-cli train \
    --stage sft \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_test,PubMedQA_pqal_test \
    --cutoff_len 1024 \
    --max_samples 100 \
    --per_device_eval_batch_size 2 \
    --predict_with_generate True \
    --max_new_tokens 512 \
    --top_p 0.7 \
    --temperature 0.95 \
    --output_dir saves\Qwen2-0.5B-Instruct\lora\eval_2024-07-09-00-25-36 \
    --do_predict True \
    --adapter_name_or_path saves\Qwen2-0.5B-Instruct\lora\train_2024-07-08-20-40-27

2024-07-11 05:39:09.551734: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 05:39:09.551804: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 05:39:09.553597: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
07/11/2024 05:39:18 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: None
[INFO|tokenization_utils_base.py:2108] 2024-07-11 05:39:18,377 >> loading file vocab.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2-0.5B-Instruct/snapshots/c291d6fce4804a1d39305f388dd32897d1f7a